In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import torch
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

In [ ]:
# Load kamitani2019 dataset (new preprocessing)

from research.data.kamitani_2019 import Kamitani2019H5

#features_model = 'biggan-128'
#features_model = 'bigbigan-resnet50'
#features_model = 'RN50'
features_model = 'ViT-B=32'
#features_model = 'vqgan'

feature_key = 'embedding'
#feature_key = 'z_mean'
#feature_key = 'vqgan-f16-1024-pre_quant'
#feature_key = 'vqgan-f16-1024-indices'

feature_selection_key = feature_key
#feature_selection_key = 'vqgan-f16-1024-pre_quant'

root = "X:\\Datasets\\Deep-Image-Reconstruction\\derivatives"
ssd_root = "C:\\Datasets\\Deep-Image-Reconstruction\\derivatives"
h5_path = Path(ssd_root) / "kamitani2019-cached.hdf5"
features_path = Path(root) / "features" / f"{features_model}-features.hdf5"
feature_selection_path = Path(root) / 'feature-selection-maps.hdf5'
subject = 'sub-03'

num_features = 2500
dataset_params = dict(
    h5_path=h5_path,
    subjects=[subject,],
    features_path=features_path,
    feature_keys=[feature_key],
    feature_selection_path=feature_selection_path,
    feature_selection_key=f'natural_training/average-4/{features_model}/{feature_selection_key}/mean-top-5',
    feature_selection_top_k=num_features,
)

dataset_training = Kamitani2019H5(
    **dataset_params,
    cached_preprocessing_name='average-4',
    func_sessions=['natural_training'],
    average_stimulus_repetitions=False,
)

dataset_test = Kamitani2019H5(
    **dataset_params,
    cached_preprocessing_name='average-4',
    func_sessions=['natural_test'],
    average_stimulus_repetitions=True,
)

dataset_imagery = Kamitani2019H5(
    **dataset_params,
    cached_preprocessing_name='average-8',
    func_sessions=['imagery'],
    average_stimulus_repetitions=True,
)

len(dataset_training), len(dataset_test), len(dataset_imagery)

In [ ]:
print([s[0]['stimulus_id'] for s in dataset_imagery.stimulus_grouped_events])

In [ ]:
data_training = dataset_training.get_data()
data_test = dataset_test.get_data()
data_imagery = dataset_imagery.get_data()
X_train = data_training['data'].float()
Y_train = data_training['features'][feature_key]
X_test = data_test['data'].float()
Y_test = data_test['features'][feature_key]
X_imagery = data_imagery['data'].float()
Y_imagery = data_imagery['features'][feature_key]

In [ ]:
# Reshape indices
Y_train, Y_test, Y_imagery = [Y.reshape(-1, 14, 14) for Y in (Y_train, Y_test, Y_imagery)]

In [ ]:
print([(item.shape, item.dtype) for item in (X_train, Y_train, X_test, Y_test, X_imagery, Y_imagery)])

In [ ]:
from einops import rearrange

embedding_weight = np.load('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\vqgan-model1024-embedding-weight.npy')
embedding_weight = torch.from_numpy(embedding_weight).cuda()
#Y = Y_test[:2].cuda()

#print(Y.shape, embedding_weight.shape)

#sqr_dist = ((Y[:, None] - embedding_weight[None, :, :, None, None]) ** 2).sum(dim=2)

In [ ]:
sqr_dist2 = (Y ** 2).sum(dim=1)[:, None] + \
            (embedding_weight ** 2).sum(dim=1)[None, :, None, None] - \
            2. * torch.einsum('bchw, ec -> behw', Y, embedding_weight)
(sqr_dist - sqr_dist2).min()

In [ ]:
# preprocessing VQGAN embedding
from pathlib import Path
import torch.nn.functional as F
from tqdm.notebook import tqdm
import gc

embedding_weight = np.load('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\vqgan-model1024-embedding-weight.npy')
embedding_weight = torch.from_numpy(embedding_weight).cuda()

def preprocess_embedding(Y):
    N, C, H, W = Y.shape
    E = embedding_weight.shape[0]
    
    Y = Y.cuda()
    Y_out = torch.zeros(N, E, H, W)

    for i in tqdm(range(Y.shape[0])):
        y = Y[i]
        y = embedding_weight[:, :, None, None] - y[None]
        y = y * y
        y = y.sum(dim=1)
        y = F.softmin(0.5 * y, dim=0)
        Y_out[i] = y.cpu()
            
    return Y_out

Y_train = preprocess_embedding(Y_train)
Y_test = preprocess_embedding(Y_test)
Y_imagery = preprocess_embedding(Y_imagery)

In [ ]:
import torch.nn.functional as F

Y = Y.sort(dim=1).values
Y = Y[:, -5:]

Y[:, -1].std()

In [ ]:
feature_shape = Y_train.shape[1:]
C, H, W = feature_shape
Y_train = Y_train.reshape(Y_train.shape[0], C * H * W)
Y_test = Y_test.reshape(Y_test.shape[0], C * H * W)
Y_imagery = Y_imagery.reshape(Y_imagery.shape[0], C * H * W)

In [ ]:
import scipy

def pearsonr(Y, Y_pred, axis=0):
    Y = Y.astype(np.double)
    Y_pred = Y_pred.astype(np.double)
    
    Y = Y - Y.mean(axis=axis, keepdims=True)
    Y_pred = Y_pred - Y_pred.mean(axis=axis, keepdims=True)
    
    Y = Y / scipy.linalg.norm(Y, axis=axis,  keepdims=True)
    Y_pred = Y_pred / scipy.linalg.norm(Y_pred, axis=axis,  keepdims=True)
    
    return (Y * Y_pred).sum(axis=axis).mean()

def cosine_similarity(Y, Y_pred, axis=0):
    Y = Y.astype(np.double)
    Y_pred = Y_pred.astype(np.double)
    
    Y = Y / scipy.linalg.norm(Y, axis=axis, keepdims=True)
    Y_pred = Y_pred / scipy.linalg.norm(Y_pred, axis=axis,  keepdims=True)
    
    return (Y * Y_pred).sum(axis=axis).mean()

def pearsonr_scipy(Y, Y_pred):
    r = []
    for i in range(Y.shape[1]):
        r.append(scipy.stats.pearsonr(Y[:, i], Y_pred[:, i])[0])
    return np.mean(r)
    
def r2_score(Y, Y_pred):
    ssr = ((Y - Y_pred) ** 2).sum(axis=0)
    sst = ((Y - Y.mean(axis=0, keepdims=True)) ** 2).sum(axis=0)
    return (1 - ssr / sst).mean()

In [ ]:
# Fit ridge regression with the FastL2LiR from the Kamitani group.
from sklearn.metrics import r2_score
from fastl2lir import FastL2LiR

model = FastL2LiR()
model.fit(X_train, Y_train, alpha=100.0, n_feat=500)

Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

print('r-batch', pearsonr(Y_train, Y_train_pred), pearsonr(Y_test, Y_test_pred))
print('r-row', pearsonr(Y_train, Y_train_pred, axis=1), pearsonr(Y_test, Y_test_pred, axis=1))
print('cs-batch', cosine_similarity(Y_train, Y_train_pred), cosine_similarity(Y_test, Y_test_pred))
print('cs-row', cosine_similarity(Y_train, Y_train_pred, axis=1), cosine_similarity(Y_test, Y_test_pred, axis=1))

In [ ]:
# Load vqgan embedding

embedding_root = 'X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\vqgan-embeddings\\'
embedding_name = 'isomap'
ndim = 8

embedding = torch.from_numpy(np.load(Path(embedding_root) / f'ndim-{ndim}' / f'{embedding_name}-embedding.npy')).float()
E = embedding.shape[1]
print(embedding.mean(), embedding.std())
embedding = (embedding - embedding.mean()) / embedding.std()
print(embedding.mean(), embedding.std())
print(embedding.shape)

Y_train, Y_train_indices = embedding[Y_train.long()], Y_train
Y_test, Y_test_indices = embedding[Y_test.long()], Y_test
Y_imagery, Y_imagery_indices = embedding[Y_imagery.long()], Y_imagery

Y_train, Y_test, Y_imagery = (Y.reshape(-1, E, 14, 14) for Y in (Y_train, Y_test, Y_imagery))

In [ ]:
# load vggan pca

from joblib import dump, load
from pathlib import Path
from einops import rearrange
from research.metrics.metrics import embedding_distance

vqgan_pca_path = Path('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\vqgan-pca\\')
n_components = 4

embedding_weight = torch.from_numpy(np.load('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\vqgan-model1024-embedding-weight.npy'))

pca = load(vqgan_pca_path / f'pca-{n_components}dim-sklearn.pkl')
def pca_transform(Y):
    N, C, H, W = Y.shape
    Y = rearrange(Y, 'n c h w -> (n h w) c')
    Y = torch.from_numpy(pca.transform(Y)).float()
    Y = rearrange(Y, '(n h w) c -> n c h w', h=H, w=W)
    return Y

def pca_inverse_transform(Y):
    N, C, H, W = Y.shape
    Y = rearrange(Y, 'n c h w -> (n h w) c')
    Y = torch.from_numpy(pca.inverse_transform(Y)).float()
    Y = rearrange(Y, '(n h w) c -> n c h w', h=H, w=W)
    return Y

#Y_test_pca = pca_transform(Y_test)
#Y_test_pca_inverse = pca_inverse_transform(Y_test_pca)

#Y_test_embedding_distance = embedding_distance(Y_test, embedding_weight)
#Y_test_pca_inverse_emebdding_distance = embedding_distance(Y_test_pca_inverse.float(), embedding_weight)

Y_train = pca_transform(Y_train).float()
Y_test = pca_transform(Y_test).float()
Y_imagery = pca_transform(Y_imagery).float()

In [ ]:
from torch.utils.data import TensorDataset

dataset_training = TensorDataset(X_train, Y_train)
dataset_test = TensorDataset(X_test, Y_test)
dataset_imagery = TensorDataset(X_imagery, Y_imagery)

In [ ]:
# Deep learning
from typing import Tuple, Optional, Dict

import torch
from torch import nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import DataLoader

from research.models.components_2d import BlurConvTranspose2d
from research.models.fmri_decoders import VariationalDecoder, SpatialDecoder, SpatialDiscriminator
from research.metrics.loss_functions import (
    EuclideanLoss, 
    EmbeddingClassifierLoss,
    ProbabalisticCrossEntropyLoss,
    VariationalLoss,
    CosineSimilarityLoss,
    EmbeddingDistributionLoss,
    ContrastiveCosineSimilarityLoss,
)
from research.metrics.metrics import cosine_similarity, pearsonr, embedding_distance


hidden_size = 512
#out_size = 1024
out_size = Y_train.shape[1]

class SpatialDecoder2(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        
        self.decoder = nn.ConvTranspose2d(in_features, out_features, kernel_size=14)
        
    def forward(self, x):
        x = x[:, :, None, None]
        x = self.decoder(x)
        return x
        

#decoder_class=SpatialDecoder,
#decoder_params={
#   'in_features': num_features, 
#   'out_features': out_size * 2, 
#   'hidden_size': hidden_size, 
#   'kernel_size': 7,
#}

#decoder_class = SpatialDecoder2
#decoder_params = {
#   'in_features': num_features, 
#   'out_features': out_size * 2, 
#}

#model = VariationalDecoder(
#    num_features, out_size, hidden_size, 
#    decoder_class=decoder_class,
#    decoder_params=decoder_params,
#)

#model = SpatialDecoder(num_features, out_size, hidden_size, kernel_size=7)

model = nn.Sequential(
    nn.Linear(in_features=num_features, out_features=hidden_size),
    nn.LeakyReLU(),
    torch.nn.Dropout(p=0.9, inplace=False),
    nn.Linear(hidden_size, out_size),
)

criterion = ContrastiveCosineSimilarityLoss()
#criterion = nn.MSELoss()
#riterion = nn.L1Loss()
#criterion = VariationalLoss(distance_loss=criterion, kld_weight=1e-2,)
#criterion = ProbabalisticCrossEntropyLoss(smoothness=0.0,)
#criterion = EuclideanLoss(dim=1)

#embedding_weight = np.load('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\vqgan-model1024-embedding-weight.npy')
#embedding_weight = torch.from_numpy(embedding_weight).cuda()
#criterion = EmbeddingDistributionLoss(embedding_weight, distance_criterion=criterion, kl_weight=.1)
#criterion = EmbeddingClassifierLoss(embedding_weight, kl_weight=1.)

optimizer = Adam(
    params=[*model.parameters(), *criterion.parameters()],
    lr=1e-4
)

device = torch.device('cuda')
model.to(device)

In [ ]:
with torch.no_grad():
    y = model(X_train[:3].cuda())
    print([e.shape for e in y])

In [ ]:
_, indices = torch.unique(Y_train, dim=0, return_inverse=True)
_, indices = torch.unique(indices[:60], return_inverse=True)
indices

In [ ]:
import torch.nn.functional as F

discriminator = SpatialDiscriminator(in_features=out_size, hidden_size=512, kernel_size=7)
discriminator_optimizer = Adam(params=discriminator.parameters(), lr=1e-5,)
discriminator.to(device)

real = 1.
fake = -1.

real_target = torch.tensor([real, fake]).cuda()[None]
fake_target = torch.tensor([fake, real]).cuda()[None]

class SoftplusLoss(nn.Module):
    def forward(self, prediction, target):
        return F.softplus(target * prediction).mean()
    
discriminator_criterion = SoftplusLoss()

discriminator_weight = 0.01

classifier = True

In [ ]:
x = torch.randn(3, 2500).cuda()
with torch.no_grad():
    x = model(x)
    print(x.shape)
    x = discriminator(x)
    print(x.shape)

In [ ]:
# Standard training

training_dataloader = DataLoader(dataset_training, shuffle=True, batch_size=1200)

def get_data_iterator(loader):
    while True:
        for batch in loader:
            yield batch
            
def run_all(dataset):
    return torch.cat([model(x.to(device)[None]).cpu() for x, _ in dataset])

training_data_iterator = get_data_iterator(training_dataloader)

max_iterations = 251
for i in range(max_iterations):
    x, y = next(training_data_iterator)
    x = x.to(device)
    y = y.to(device)

    model.train()
    y_pred = model(x)
    #y = y.reshape(y_pred.shape)
    loss = criterion(y, y_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    
    if i % 25 == 0:
        print(round(loss.detach().cpu().item(), 3))
    
    if i % 250 == 0:
        model.eval()
        with torch.no_grad():
            Y_train_pred = run_all(dataset_training)
            Y_test_pred = run_all(dataset_test)
            Y_train = dataset_training.tensors[1]
            Y_test = dataset_test.tensors[1]
        
        print('r-batch', 
              pearsonr(Y_train, Y_train_pred), 
              pearsonr(Y_test, Y_test_pred))
        print('cs-row', 
              cosine_similarity(Y_train, Y_train_pred, dim=1), 
              cosine_similarity(Y_test, Y_test_pred, dim=1))

In [ ]:
def cosine_similiarity2(Y, Y_pred):
    Y = Y.to(torch.float64)
    Y_pred = Y_pred.to(torch.float64)

    Y = Y / torch.norm(Y, dim=-1, keepdim=True)
    Y_pred = Y_pred / torch.norm(Y_pred, dim=-1, keepdim=True)
    
    return Y @ Y_pred.T

cs = cosine_similiarity2(Y_test, Y_test_pred)
print(((cs.argsort(dim=0).diag() / cs.shape[0]).mean() + (cs.argsort(dim=1).diag() / cs.shape[0]).mean()) * 0.5)
plt.imshow(cs)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.random.randn(8, 8)
plt.axis('off')
plt.imshow(x, cmap='gray')

In [ ]:
cs.shape

In [ ]:
Y_test.shape

In [ ]:
# classifier training

Y_train = dataset_training.tensors[1]
Y_test = dataset_test.tensors[1]

training_dataloader = DataLoader(dataset_training, shuffle=True, batch_size=24)

def get_data_iterator(loader):
    while True:
        for batch in loader:
            yield batch
            
def run_all(dataset):
    return torch.cat([model(x.to(device)[None]).cpu() for x, _ in dataset])

training_data_iterator = get_data_iterator(training_dataloader)

max_iterations = 251
for i in range(max_iterations):
    x, y = next(training_data_iterator)
    x = x.to(device)
    y = y.to(device)

    model.train()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if isinstance(loss, tuple):
        loss, loss_dict = loss
    else:
        loss_dict = loss.detach().cpu().item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    
    if i % 10 == 0:
        print('train loss', loss_dict)
    
    if i % 50 == 0:
        model.eval()
        with torch.no_grad():
            pass
            #Y_train_pred = run_all(dataset_training)
            Y_test_pred = run_all(dataset_test)
            
            print(f'test loss {criterion(Y_test_pred, Y_test)}')

In [ ]:
# Adversarial training

training_dataloader = DataLoader(dataset_training, shuffle=True, batch_size=32)

def get_data_iterator(loader):
    while True:
        for batch in loader:
            yield batch
            
def run_all(dataset):
    return torch.cat([model(x.to(device)[None]).cpu() for x, _ in dataset])

training_data_iterator = get_data_iterator(training_dataloader)

def fmt_loss(loss):
    return round(loss.detach().cpu().item(), 3)

max_iterations = 251
for i in range(max_iterations):
    x, y = next(training_data_iterator)
    x = x.to(device)
    y = y.to(device)
    
    if classifier:
        y = embedding_distance(y, embedding_weight.cuda())
        y = y ** 0.5
        y = F.softmax(-y, dim=1)
    
    model.eval()
    model.zero_grad()
    discriminator.train()
    discriminator.zero_grad()
    
    d_real_pred = discriminator(y)
    d_fake_pred = discriminator(model(x))
    d_real_loss = discriminator_criterion(d_real_pred, real_target)
    d_fake_loss = discriminator_criterion(d_fake_pred, fake_target)
    d_loss = (d_real_loss + d_fake_loss) * 0.5
    
    discriminator_optimizer.zero_grad()
    d_loss.backward()
    discriminator_optimizer.step()

    model.train()
    model.zero_grad()
    discriminator.eval()
    discriminator.zero_grad()
    
    y_pred = model(x)
    loss = criterion(y, y_pred)
    
    g_fake_pred = discriminator(y_pred)
    g_fake_loss = discriminator_criterion(g_fake_pred, real_target)
    if i % 10 == 0:
        print(f'iter={i}, structural_loss={fmt_loss(loss)}, g_fake_loss={fmt_loss(g_fake_loss)}, d_real_loss={fmt_loss(d_real_loss)}, d_fake_loss={fmt_loss(d_fake_loss)}')
    loss = loss + g_fake_loss * discriminator_weight

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    
    if i % 250 == 0:
        model.eval()
        with torch.no_grad():
            Y_train_pred = run_all(dataset_training)
            Y_test_pred = run_all(dataset_test)
            Y_train = dataset_training.tensors[1]
            Y_test = dataset_test.tensors[1]
        
        print('r-batch', 
              pearsonr_torch(Y_train, Y_train_pred), 
              pearsonr_torch(Y_test, Y_test_pred))
        print('cs-row', 
              cosine_similarity_torch(Y_train, Y_train_pred, dim=1), 
              cosine_similarity_torch(Y_test, Y_test_pred, dim=1))

In [ ]:
# Variational training

training_dataloader = DataLoader(dataset_training, shuffle=True, batch_size=128)

def get_data_iterator(loader):
    while True:
        for batch in loader:
            yield batch
            
def run_all(dataset):
    return torch.cat([model(x.to(device)[None])[0] for x, _ in dataset]).cpu()

training_data_iterator = get_data_iterator(training_dataloader)

max_iterations = 1500
for i in range(max_iterations):
    x, y = next(training_data_iterator)
    x = x.to(device)
    y = y.to(device)

    model.train()
    y_pred, mu, log_var = model(x)
    loss, loss_dict = criterion(y, y_pred, mu, log_var)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    
    if i % 25 == 0:
        print({k: round(v.detach().cpu().item(), 3) for k, v in loss_dict.items()})
    
    if i % 250 == 0:
        model.eval()
        with torch.no_grad():
            Y_train_pred = run_all(dataset_training)
            Y_test_pred = run_all(dataset_test)
        
        print('r-batch', 
              pearsonr(dataset_training.tensors[1], Y_train_pred), 
              pearsonr(dataset_test.tensors[1], Y_test_pred))
        print('cs-row', 
              cosine_similarity(dataset_training.tensors[1], Y_train_pred, dim=1), 
              cosine_similarity(dataset_test.tensors[1], Y_test_pred, dim=1))

In [ ]:
from pathlib import Path

out_path = Path('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\')

features_model

np.save(out_path / f'{features_model}__{Y_key}__{subject}__test-prediction__v2.npy', Y_scaler.inverse_transform(Y_test_pred).reshape(50, 128))#
np.save(out_path / f'{features_model}__{Y_key}__{subject}__test__v2.npy', Y_scaler.inverse_transform(Y_test).reshape(50, 128))#*Y_shape))

In [ ]:
subject

In [ ]:
from pathlib import Path

out_path = Path('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\decoded_features')

np.save(out_path / f'{features_model}__{feature_key}__{subject}__test-prediction__v4.npy', Y_test_pred)#
np.save(out_path / f'{features_model}__{feature_key}__{subject}__test__v4.npy', Y_test)#*Y_shape))

In [ ]:
Y_test_pred[:, 1].mean()

In [ ]:
from pathlib import Path

out_path = Path('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\decoded_features')

def run_all_variational(dataset):
    mean = []
    std = []
    for x, _ in dataset:
        _, mu, log_var = model(x.to(device)[None])
        mean.append(mu)
        std.append(torch.exp(0.5 * log_var))
    mean = torch.cat(mean)
    std = torch.cat(std)
    out = torch.stack([mean, std], dim=1)
    return out.detach().cpu().numpy()
        
Y_test_pred = run_all_variational(dataset_test)
Y_imagery_pred = run_all_variational(dataset_imagery)

version = '4-0'
code_name = 'embedding'
out_dir = out_path / features_model / feature_key / subject
out_dir.mkdir(parents=True, exist_ok=True)
np.save(out_dir / f'natural_test__{code_name}__v{version}.npy', Y_test_pred)#.reshape(Y_test_pred.shape[0], *feature_shape))
np.save(out_dir / f'imagery__{code_name}__v{version}.npy', Y_imagery_pred)#.reshape(Y_imagery_pred.shape[0], *feature_shape))

if code_name == 'embedding':
    np.save(out_dir / f'{code_name}__v{version}.npy', embedding.cpu().numpy())#.reshape(Y_imagery_pred.shape[0], *feature_shape))

In [ ]:
embedding.shape

In [ ]:
from pathlib import Path

out_path = Path('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\decoded_features')

with torch.no_grad():
    Y_test_pred = run_all(dataset_test).cpu().numpy()
    Y_imagery_pred = run_all(dataset_imagery).cpu().numpy()

version = '1-0'
code_name = 'contrastive'
out_dir = out_path / features_model / feature_key / subject
out_dir.mkdir(parents=True, exist_ok=True)
np.save(out_dir / f'natural_test__{code_name}__v{version}.npy', Y_test_pred)#.reshape(Y_test_pred.shape[0], *feature_shape))
np.save(out_dir / f'imagery__{code_name}__v{version}.npy', Y_imagery_pred)#.reshape(Y_imagery_pred.shape[0], *feature_shape))

if code_name == 'embedding':
    np.save(out_dir / f'{code_name}__v{version}.npy', embedding.cpu().numpy())#.reshape(Y_imagery_pred.shape[0], *feature_shape))

In [ ]:
from pathlib import Path

out_path = Path('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\decoded_features')

with torch.no_grad():
    Y_imagery_pred = run_all(dataset_imagery).cpu().numpy()


version = '1-0'
code_name = 'pca'
out_dir = out_path / features_model / feature_key / subject
out_dir.mkdir(parents=True, exist_ok=True)
np.save(out_dir / f'natural_test__{code_name}__v{version}.npy', pca_inverse_transform(Y_test_pred))
np.save(out_dir / f'imagery__{code_name}__v{version}.npy', pca_inverse_transform(Y_imagery_pred))#.reshape(Y_imagery_pred.shape[0], *feature_shape))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm

pipelines = []
for i in tqdm(range(Y_train.shape[1])):
    pipeline = Pipeline([
        ('feature_selection', SelectKBest(f_classif, k=500)),
        ('model', LogisticRegression(solver='liblinear'))
    ])
    pipeline.fit(X_train, Y_train[:, i])
    pipelines.append(pipeline)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from tqdm.notebook import tqdm

select_k_best = SelectKBest(f_classif, k=5)
X_train_selection = select_k_best.fit_transform(X_train, Y_train[:, 0])

model = LogisticRegression(solver='saga', C=1000.0)
model.fit(X_train_selection, Y_train[:, 0])

X_test_selection = select_k_best.transform(X_test,)
print(model.score(X_train_selection, Y_train[:, 0]), model.score(X_test_selection, Y_test[:, 0]))

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

df = pd.read_pickle('G:\\Github Repositories\\GenericObjectDecoding\\code\\python\\results\\GenericObjectDecoding.pkl')
#print(df)
df = df[df['feature'] == 'cnn8']

true_feature_averaged_percept = df['true_feature_averaged_percept'].array[0]
predicted_feature_averaged_percept = df['predicted_feature_averaged_percept'].array[0]
print(predicted_feature_averaged_percept.shape)

print(r2_score(true_feature_averaged_percept, predicted_feature_averaged_percept))
print(pearsonr(true_feature_averaged_percept, predicted_feature_averaged_percept))

In [ ]:
Y_stuff = Y_train.reshape(4800, *Y_shape)
Y_stuff.std(axis=(0, 1))